In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import cv2


Mount drive (left sidebar) and use the proper dir

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [16]:
!unzip /content/drive/MyDrive/HCCR/wb_recognition_dataset.zip &> /dev/null

In [3]:
dataset_dir = 'wb_recognition_dataset'
train_dataset = dataset_dir + '/train'
val_dataset = dataset_dir + '/val'

In [12]:
# %cd train_dataset
image_folders = os.listdir(train_dataset)
print(image_folders)

['1863', '1793', '2136', '2113', '1474', '2028', '113', '1610', '415', '1439', '1238', '588', '308', '1325', '1526', '2060', '1492', '1534', '1174', '1223', '927', '463', '1611', '1655', '276', '673', '2122', '1797', '608', '1265', '1615', '1547', '893', '897', '1330', '175', '86', '811', '760', '1882', '370', '580', '1500', '833', '816', '290', '1919', '62', '1309', '1271', '1660', '1770', '2130', '2119', '1349', '300', '128', '1957', '294', '1052', '481', '237', '851', '1932', '114', '1834', '1540', '1088', '750', '1870', '1754', '1812', '506', '356', '879', '1429', '695', '609', '1197', '485', '404', '629', '1856', '1651', '1820', '122', '1938', '762', '1731', '975', '1013', '193', '613', '1600', '1097', '1796', '171', '372', '1663', '616', '707', '1504', '520', '738', '426', '1623', '2104', '984', '518', '1711', '345', '996', '970', '2055', '1280', '802', '1024', '1980', '888', '1496', '330', '1180', '1204', '461', '697', '535', '151', '1835', '1141', '293', '2018', '1730', '941', 

In [13]:
def load_images_from_folder(folder) -> tuple:
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)

    label = os.path.basename(folder)
    return label, images

In [14]:
train_imgs = []
train_labels = []
for folder_name in image_folders:
    dis_dir = f"{train_dataset}/{folder_name}"
    label, images = load_images_from_folder(dis_dir)

    for img in images:
        train_labels.append(label)
        train_imgs.append(img)